06　Affine・Softmaxレイヤの実装
===========================

## 1. Affineレイヤ

* ニューラルネットワークの順伝播では、重み付き信号の総和を計算するため、行列の積を用いた(`np.dot()`)

* 前の章では、以下のような実装を行なった

In [1]:
import numpy as np

# 入力
X = np.random.rand(2)

# 重み
W = np.random.rand(2, 3)

# バイアス
B = np.random.rand(3)

In [2]:
X.shape

(2,)

In [3]:
W.shape

(2, 3)

In [4]:
B.shape

(3,)

In [5]:
Y = np.dot(X, W) + B

* ここで、`X`、`W`、`B`はそれぞれ形状が多次元配列となっている

    * そうすることで、ニューロンの重み付き和は、`Y = np.dot(X, W) + B`のように計算できる
    
    * そして、この`Y`が活性化関数によって、変換され、次の層へと伝播される(NNにおける順伝播の流れ)

* 次に、ここで行なった計算(行列の積とバイアスの和)を計算グラフで表す

    * 行列の積を計算するノードを「`dot`」として表すことにすると、`np.dot(X, W) + B`の計算は、以下の式で表すことができる
    
![Affileレイヤの計算グラフ](./images/Affileレイヤの計算グラフ.png)

> ここでは、「行列」がノード間を伝播する

### 計算グラフの逆伝播

* 行列を対象とした逆伝播を求める場合は、計算グラフと同じ手順で考えることができる

\begin{eqnarray}
\frac{\partial L}{\partial \vec{X}} = \frac{\partial L}{\partial \vec{Y}} \cdot \vec{W^T} \\
\frac{\partial L}{\partial \vec{W}} =  \vec {X^T} \cdot\frac{\partial L}{\partial \vec{Y}}
\end{eqnarray}

* $\vec{W^T}$の$T$：転置行列

    * $\vec{W}$の$(i, j)$の要素を、$(j, i)$の要素に入れ替えることを言う
    
\begin{eqnarray}
\vec{W} = 
\begin{pmatrix}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23}
\end{pmatrix} \\
\vec{W^T} = 
\begin{pmatrix}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{pmatrix}
\end{eqnarray}

* 上の式を用いて、計算グラフの逆伝播を記述する

    * ここで、 $\vec{W}$と$\frac{\partial L}{\partial \vec{W}}$は同じ形状
    
\begin{eqnarray}
\vec{W} = (x_0, x_1, \cdots, x_n) \\
\frac{\partial L}{\partial \vec{X}} =  \Bigl(
\frac{\partial L}{\partial x_0}, \frac{\partial L}{\partial x_1}, \cdots, \frac{\partial L}{\partial x_n}
\Bigr)
\end{eqnarray}

![Affineレイヤの逆伝播](./images/Affineレイヤの逆伝播.png)

## 2. バッチ版Affineレイヤ

* ここで、N個のデータをまとめて順伝播する場合(バッチ)を考える

    * 入力である$\vec{X}$の形状が$(N, 2)$となる
    
![バッチ版Affineレイヤの計算グラフ](./images/バッチ版Affineレイヤの計算グラフ.png)

* ただし、順伝播の際のバイアスの加算は$\vec{X} \cdot \vec{W}$に対して、バイアスがそれぞれのデータに加算される
    


In [6]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

In [7]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [8]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

* 順伝播でのバイアスの加算は、それぞれのデータ(1個目のデータ、2個目のデータ、...)に対して加算が行われる

* そのため、逆伝播の際には、それぞれのデータの逆伝播の値がバイアスの要素に集約される必要がある

In [9]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [10]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

* 上の例では、データが2個(N=2)あるものと仮定する

* バイアスの逆伝播は、その2個のデータに対しての微分を、データごとに合算して求める

    * そのため、`np.sum()`で、0番目の軸(データを単位とした軸)に対して(`axis=0`)の総和を求める

* 実装は、以下の通りになる

In [11]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 重み・バイアスパラメータの微分
        self.dW = None
        self.db = None

    def forward(self, x):
        # テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx

## 3. Softmax-with-Lossレイヤ

* 出力層である`ソフトマックス関数`について扱う

    * この関数は、入力された値を正規化して出力する
    
    * 例)手書き数字認識の場合
    
![入力画像とAffine・ReLUレイヤ](./images/入力画像とAffine・ReLUレイヤ.png)

* 上の図のように、Softmaxレイヤの出力は、入力された値を正規化(出力の和が`1`になるように変形)して出力する

* ただし、手書き数字認識は、10クラス分類を行うため、Softmaxレイヤの入力は10個あることになる

> ニューラルネットワークで行う処理には、`推論`と`学習`の2つのフェーズがある
>
> `推論`：通常、Softmaxレイヤは使用しない(Affineレイヤの出力結果を認識結果として用いる)
> 
> `学習`：Softmaxレイヤが必要になる

* Softmaxレイヤを実装していくが、損失関数である`交差エントロピー誤差`も含めて、「Softmax-with-Lossレイヤ」という名前のレイヤで実装する

![Soft-with-Lossレイヤの計算グラフ](./images/Soft-with-Lossレイヤの計算グラフ.png)

* 上の図の通り、Softmax-with-Lossレイヤはやや複雑

* この計算グラフを簡略化して書くと、以下の通りになる

![簡易版Softmax-with-Lossレイヤ](./images/簡易版Softmax-with-Lossレイヤ.png)

> 上の計算グラフでは、ソフトマックス関数はSoftmaxレイヤとして、`交差エントロピー誤差`はCross Entropy Errorレイヤとして表記する

* ここでは、3クラス分類を行う場合を想定し、前レイヤから3つの入力(スコア)を受け取るものとする

* Softmaxレイヤは、入力である$(a_1, a_2, a_3)$を正規化して、$(y_1, y_2, y_3)$を出力する

* Cross Entropy Errorレイヤは、Softmaxの出力$(y_1, y_2, y_3)$と、教師ラベルの$(t_1, t_2, t_3)$を受け取り、それらのデータから損失$L$を出力する

* 上の出力で注目すべきは、逆伝播の結果

* Softmaxレイヤからの逆伝播は、$(y_1 - t_1, y_2 - t_2, y_3 - t_3)$という結果となる

    * $(y_1, y_2, y_3)$：Softmaxレイヤの出力
    
    * $(t_1, t_2, t_3)$：教師データ
    
    * そのため、これはSoftmaxレイヤの出力と教師ラベルの差分となる
    
* ニューラルネットワークの逆伝播では、この差分である誤差が前レイヤへ伝わっていく

    * これは、ニューラルネットワークの学習において重要な性質となる

* ここで、ニューラルネットワークの学習の目的は、ニューラルネットワークの出力(Softmaxの出力)を教師ラベルに近づけるように、重みパラメータを調整すること

    * そのため、ニューラルネットワークの出力と教師ラベルとの誤差を効率よく、前レイヤに伝える必要がある
    
    * そのため、この差分は現在のニューラルネットワークの出力と教師ラベルの誤差を素直に表している

> `交差エントロピー誤差`は、このように綺麗な結果になるように作られた

* ここで具体例を考える

* 例)教師ラベルが$(0,1, 0)$であるデータに対して、Softmaxレイヤの出力が$(0.3, 0.2, 0.5)$であった場合

    * 正解ラベルに対する確率は$0.2$なので、この時点のニューラルネットワークは正しい認識ができていない
    
    * この場合、Softmaxレイヤからの逆伝播は、$(0.3, -0.8, 0.5)$という大きな誤差を伝播することになる
    
    * この大きな誤差が前レイヤに伝播していくので、Softmaxレイヤよりも前のレイヤは、その大きな誤差から大きな内容を学習することになる

* 例)教師ラベルが$(0, 1, 0)$であるデータに対して、Softmaxレイヤの出力が$(0.01, 0.99, 0)$の場合

    * この場合、Softmaxレイヤからの逆伝播は、$(0.01, -0.01, 0)$ という小さなgosaninaru
    
    * この小さな誤差が前レイヤに伝播していくが、その誤差は小さいため、Softmaxレイヤよりも前にあるレイヤが学習する内容も小さくなる

* 実装は、以下の通りになる

In [12]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmaxの出力
        self.t = None # 教師データ

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 教師データがone-hot-vectorの場合
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/05/11 |